In [36]:
## conda install -c conda-forge dlib
import numpy as np
import argparse
import cv2
import dlib
import imutils
import matplotlib.pyplot as plt
from os import listdir
from os.path import join, isfile
import csv
import itertools

In [37]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    
    return (x, y, w, h)

In [38]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    return coords

In [58]:
def extract_features(keypoints): ## all possible combinations instead of 11 
    assert keypoints.shape == (68,2)
    feats = []
    denom = np.linalg.norm(keypoints[0]-keypoints[16])
    
    combs = [comb for comb in itertools.combinations([*range(0, len(keypoints)-1)], 2)]
    for comb in combs:
        a = comb[0]
        b = comb[1]
        
        if not (a ==0 and b == 16):
            feats.append(np.linalg.norm(keypoints[a]-keypoints[b])/denom)
            
    return [], feats

In [59]:
def get_features(path):
    # load the input image, resize it, and convert it to grayscale
    image = cv2.imread(path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    for (i, rect) in enumerate(rects):
        
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)

        keypoints, feats = extract_features(shape)
    
        return feats
    print("No face found")
    print(path)
    return np.zeros(11).tolist()

## Saving all images with this representation

In [60]:
detector = dlib.get_frontal_face_detector()
path_to_shape_predictor = "models/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(path_to_shape_predictor)

In [76]:
lowest_age = 17
highest_age = 40

In [77]:
# open directories
kdv_dir = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien\kdv-patients-age-group-"+str(lowest_age) + "-" + str(highest_age)
ID_dir = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien\kdv-selected-ID-controls-age-group-"+str(lowest_age) + "-" + str(highest_age)

# get list of filenames
files_kdv = [f for f in listdir(kdv_dir) if (isfile(join(kdv_dir, f)) & ("crop_sized.jpg" in f))]
files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f)) & ("crop_sized.JPG" in f))]

print(len(files_kdv))
print(len(files_ID))

9
9


In [78]:
kdv_rep = []
ID_rep = []

# for each kdv image save deepface rep as list:
for filename in files_kdv:
    feats = get_features(join(kdv_dir, filename))
    kdv_rep.append([filename] + feats)  

# for each ID image save deepface rep as list:
for filename in files_ID:
    feats = get_features(join(ID_dir, filename))
    ID_rep.append([filename] + feats)  

# location to save representation
csv_file_kdv = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien\representations\kdv-patients-dlib-cropped-age-group-"+str(lowest_age) + "-" + str(highest_age)+".csv"
csv_file_ID = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien\representations\ID-controls-dlib-cropped-age-group-"+str(lowest_age) + "-" + str(highest_age)+".csv"

# save representation of kdv patients
with open(csv_file_kdv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(kdv_rep)

# save representation of ID controls
with open(csv_file_ID, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(ID_rep)

print("Done with saving all dlib representations.")

No face found
H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien\kdv-patients-age-group-17-40\kdvs_52_crop_sized.jpg
Done with saving all dlib representations.
